In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install nlpaug
!pip install googletrans
!pip install sacremoses

In [ ]:
import re
import nltk
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Initialize augmenters
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word as naw
from googletrans import Translator
import pandas as pd
import nlpaug.flow as naf
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Text Summarization/LogDataset.csv")
dataset.head()

,Task Description,Summary
0,"1. Meeting list, task list design, set their p...",describes a series of tasks related to designi...
1,1. Food app few design research.\n2. About foo...,There are many tasks related to designing and ...
2,1. Daily work update & new requirement.\n2. EB...,daily work updates and new requirements for va...
3,Absolutely! Here is a structured list of the t...,The employee's work focused primarily on commu...
4,1. Analyzing the ThriveDesk website and docume...,"analyzed and documented several websites, incl..."


In [ ]:
dataset["Task Description"][0]

"1. Meeting list, task list design, set their position & prototype all these screens.\n2. Meeting details design & prototype all screens.\n3. Task list details design & prototype them.\n4. Upcoming, pending, postponed, cancelled meeting & task list design also prototype them.\n5. Google Material Design.\n6. Meeting list & task list redesign & also prototype them.\n7. Team meeting list, team task list, team call log design & also prototype them.\n8. Team meeting list details, team task list details, team call log details design & also prototype them.\n9. Meeting details redesign, activity log design.\n10. Video Gallery Inauguration.\n11. Activity Log, Team Activity Log design.\n12. Team Task Details, team meeting details, team log hour.\n13. Few screen prototypes.\n14. EBS Home page redesign.\n15. About EBS Home Design's few issues.\n16. Adding some features in EBS homepage.\n17. Calendar design for EBS (new feature), adding some things like team lead schedule, upcoming schedule, pendin

In [ ]:
# shuffled_dataframe = dataset.sample(frac=1).reset_index(drop=True)
shuffled_dataframe = dataset.copy()
# shuffled_dataframe.rename(columns={'Task Description': 'dialogue', 'Summary': 'summary'}, inplace=True)

In [ ]:
def clean_and_lemmatize(text):
    # Remove digits, symbols, punctuation marks, and newline characters
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text.replace('\n', ''))
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Lemmatize each token and join back into a sentence
    lemmatized_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return lemmatized_text
# Apply cleaning and lemmatization to the 'Task Description' column
shuffled_dataframe['Task Description'] = shuffled_dataframe['Task Description'].apply(clean_and_lemmatize)
shuffled_dataframe['Summary'] = shuffled_dataframe['Summary'].apply(clean_and_lemmatize)

In [ ]:
shuffled_dataframe.head()

,Task Description,Summary
0,meeting list task list design set their positi...,describes a series of task related to designin...
1,food app few design research about food app ne...,there are many task related to designing and p...
2,daily work update new requirement eb contact p...,daily work update and new requirement for vari...
3,absolutely here is a structured list of the ta...,the employee work focused primarily on communi...
4,analyzing the thrivedesk website and documenti...,analyzed and documented several website includ...


In [ ]:
shuffled_dataframe["Task Description"][0]

'meeting list task list design set their position prototype all these screen meeting detail design prototype all screen task list detail design prototype them upcoming pending postponed cancelled meeting task list design also prototype them google material design meeting list task list redesign also prototype them team meeting list team task list team call log design also prototype them team meeting list detail team task list detail team call log detail design also prototype them meeting detail redesign activity log design video gallery inauguration activity log team activity log design team task detail team meeting detail team log hour few screen prototype eb home page redesign about eb home design few issue adding some feature in eb homepage calendar design for eb new feature adding some thing like team lead schedule upcoming schedule pending schedule cancel schedule today schedule weekly schedule last day schedule monthly schedule calendar design for eb new feature here also add tea

In [ ]:
# Initialize augmenters
aug_synonym = naw.SynonymAug(aug_src='wordnet', aug_p = 0.4)
aug_insert = naw.RandomWordAug(aug_p = 0.3)
# Function to perform all augmentations
def perform_augmentations(text):
    # Synonym Augmentation
    text = aug_synonym.augment(text)

    # Random Word Insertion
    text = aug_insert.augment(text)

    return text

In [ ]:
augmented_data = []
for index, row in shuffled_dataframe.iterrows():
    for _ in range(5):
        augmented_desc = perform_augmentations(row['Task Description'])
        augmented_summ = perform_augmentations(row['Summary'])
        augmented_data.append({'Task Description': augmented_desc, 'Summary': augmented_summ})

In [ ]:
# Creating a DataFrame with augmented text
augmented_df = pd.DataFrame(augmented_data)
# Save augmented data to a CSV file
augmented_df.to_csv('augmented_Dataset_200_data.csv', index=False)

In [ ]:
pd.set_option('display.max_colwidth', -1)
aug_dataset = pd.read_csv("/content/augmented_Dataset_200_data.csv")
aug_dataset.head()

<ipython-input-13-80520ec84b5e>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Task Description  \
0  ['meeting list task list design set their position prototype all these screen meeting detail design prototype all screen task list detail design prototype them upcoming pending postponed cancelled meeting task list design also prototype them design meeting list task list redesign also prototype them team meeting list team task list squad birdcall log design also prototype them team meeting list detail team task list detail team call log detail design also prototype them meeting detail redesign activity log design video gallery inauguration activity log team activity log design team task detail team meeting detail team log hour few screen prototype eb home varlet redesign about eb home design few issue adding some feature in eb homepage calendar design for eb new feature adding some thing like team lead schedule upcoming schedule pending schedule cancel schedule today schedule weekly schedule last day schedule monthly schedule calendar design for eb new feature here also add team schedule upcoming schedule pending schedule cancel schedule today schedule weekly schedule last day schedule monthly schedule eb design confirmation new requirement learning design concept eb calendar design design concept for better ui design calendar pending task design some unnecessary feature removed and a few screen prototype eb calendar pending task design change eb design purpose meeting about invideo website design research for leave application in eb eb leave application design two leave application demo design home page upcoming pending demo change design prototype them learn a few design concept for better app ui design research a few design for eb better ui online research for leave application ui design research design implementation and few change in design leave detail design prototype screen eb leave application ui design prototype them eb leave application ui design change new requirement design eb leave calendar design change research in food app meeting about food app food app flow chart research and flow chart design few food for thought app ui research eb leave application design change prototype them meeting about leave application design meeting about eb leave application design final eb homepage design change prototype them eb leave application design concept briefing for production eb home page design again change prototype with other screen learning research for uiux presentation making of a presentation slide watching video for better ui design reading few case study of many uiux project practicing a few design eb leave detail few thing in design are change few thing research for uiux presentation full presentation complete on uiux learn research a few thing for better ui design leave application all screen prototype research design a user flow for food saver app meeting about food saver app user flow change a few flow chart element and creates a new flow chart for the food saver app about food saver app design design an algorithm flow chart for egov team full food saver app flowchart pattern implement in figma eb calendar design few thing change research few food app a few concept selected for food saver app learn a few thing for better ui about food app food saver app flow chart full change food saver app flow chart few modification and new feature add uiux working process presentation discus food saver app flow chart research few presentation for building a good presentation make presentation done on uiux working process about food saver app flowchart eb calendar design change eb calendar new feature implement eb attendance overview flow chart color research change give it to production research a few thing for food app ui research on eb meeting task call log activity log to reduce form size also make a presentation on it eb home page full ui alteration eb all contact list ui change eb design after implementation full checking eb home page ui design change eb all icon change eb homepage after product

In [ ]:
aug_dataset["Task Description"][0]

"['meeting list task list design set their position prototype all these screen meeting detail design prototype all screen task list detail design prototype them upcoming pending postponed cancelled meeting task list design also prototype them design meeting list task list redesign also prototype them team meeting list team task list squad birdcall log design also prototype them team meeting list detail team task list detail team call log detail design also prototype them meeting detail redesign activity log design video gallery inauguration activity log team activity log design team task detail team meeting detail team log hour few screen prototype eb home varlet redesign about eb home design few issue adding some feature in eb homepage calendar design for eb new feature adding some thing like team lead schedule upcoming schedule pending schedule cancel schedule today schedule weekly schedule last day schedule monthly schedule calendar design for eb new feature here also add team sched

In [ ]:
aug_dataset.shape

(175, 2)

In [ ]:
import os
dataframes = []
data_file = ["/content/drive/MyDrive/Text Summarization/LogDataset.csv", "/content/augmented_Dataset_200_data.csv"]
for file in data_file:
  # Read the CSV file into a DataFrame
  data = pd.read_csv(file)

  # Append the data to the dataframes list
  dataframes.append(data)

# Concatenate all DataFrames in the list into a single DataFrame
concatenated_data = pd.concat(dataframes, ignore_index=True)

# Save the concatenated data to a new CSV file
concatenated_data.to_csv('Dataset.csv', index=False)


In [ ]:
data = pd.read_csv("/content/Dataset.csv")
data.head()

Task Description  \
0  1. Meeting list, task list design, set their position & prototype all these screens.\n2. Meeting details design & prototype all screens.\n3. Task list details design & prototype them.\n4. Upcoming, pending, postponed, cancelled meeting & task list design also prototype them.\n5. Google Material Design.\n6. Meeting list & task list redesign & also prototype them.\n7. Team meeting list, team task list, team call log design & also prototype them.\n8. Team meeting list details, team task list details, team call log details design & also prototype them.\n9. Meeting details redesign, activity log design.\n10. Video Gallery Inauguration.\n11. Activity Log, Team Activity Log design.\n12. Team Task Details, team meeting details, team log hour.\n13. Few screen prototypes.\n14. EBS Home page redesign.\n15. About EBS Home Design's few issues.\n16. Adding some features in EBS homepage.\n17. Calendar design for EBS (new feature), adding some things like team lead schedule, upcoming schedule, pending schedule, Cancel schedule, today schedule, weekly schedule, last 3 days schedule, monthly schedule.\n18. Calendar design for EBS (new feature), here also add team schedule, upcoming schedule, pending schedule, cancel schedule, today schedule, weekly schedule, last 3 days schedule, monthly schedule.\n19. EBS design confirmation & new requirement.\n20. Learning design concept.\n21. EBS Calendar Design.\n22. Learning design concepts for better UI design.\n23. Calendar pending task design, some unnecessary features removed, and a few screen prototypes.\n24. EBS Calendar pending task Design change.\n25. EBS design purpose meeting.\n26. About Invideo website.\n27. Design research for leave applications in EBS.\n28. EBS Leave application design, two leave application demo design, home page upcoming & pending demo change design & prototype them.\n29. Learn a few design concepts for better app UI design.\n30. Research a few designs for EBS better UI.\n31. Online research for leave application UI design.\n32. Research design implementation and few changes in design.\n33. Leave details design & prototype screens.\n34. EBS Leave application UI design & prototype them.\n35. EBS Leave application UI design Change & new requirement design.\n36. EBS leave calendar design change.\n37. Research in food app.\n38. Meeting about food app.\n39. Food app flow chart research and flow chart design.\n40. Few food app UI research.\n41. EBS Leave application design change & prototype them.\n42. Meeting about Leave application design.\n43. Meeting about EBS leave application design final.\n44. EBS Homepage design change & prototype them.\n45. EBS leave application design concept briefing for production.\n46. EBS homepage design again change & prototype with other screens.\n47. Learning & Research for a UI/UX presentation.\n48. Making 50% of a presentation slide.\n49. Watching videos for better UI design.\n50. Reading few case studies of many UI/UX projects & practicing a few designs.\n51. EBS leave details few things in design which are change.\n52. Few things research for UI/UX presentation.\n53. Full presentation complete on UI/UX.\n54. Learn & research a few things for better UI design.\n55. EBS leave application all screen prototype.\n56. Research & Design a user flow for Food Saver app.\n57. Meeting about Food saver app user flow.\n58. Changes a few flow chart elements and creates a new flow chart for the food saver app.\n59. About food saver app design.\n60. Design an algorithm flow chart for e-gov team.\n61. Full food saver app flowchart design implement in Figma.\n62. EBS Calendar design few things changes.\n63. Research few food app & a few concepts selected for food saver app.\n64. Learn a few things for better UI.\n65. About food app.\n66. Food Saver App flow chart full change.\n67. Food Saver app flow chart few modifications and new features add.\n68. UI/UX Working process presentation & discuss food saver app flow chart.\n69. Research few

In [ ]:
data.shape

(210, 2)

In [ ]:
data.duplicated().sum()

0